In [1]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer as TF
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from conf import config

In [2]:
# train = pd.read_csv(config.train_path)
train = pd.read_csv('../data/new_train.csv')
test = pd.read_csv(config.test_path)

In [3]:
train['review'].head()

0    stuff going moment mj started listening music ...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

In [4]:
tfidf = TF(analyzer="word",
           tokenizer=None,
           preprocessor=None,
           stop_words=None,
           max_features=5000)

# 数据向量化
print("Creating the tfidf vector...\n")
tfidf.fit(train['review'])
x_train = tfidf.transform(train['review'])
x_train = x_train.toarray()

x_test = tfidf.transform(test['review'])
x_test = x_test.toarray()

print(x_train.shape)
print(x_test.shape)

Creating the tfidf vector...

(75000, 5000)
(25000, 5000)


In [5]:
model = LR(solver='liblinear')
model.fit(x_train, train["sentiment"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [6]:
print("10折交叉验证：")
print(np.mean(cross_val_score(model, x_train, train["sentiment"], cv=10, scoring="roc_auc")))

10折交叉验证：
0.9617291946666666


In [7]:
# preds = model.predict(x_test)

In [8]:
# submission=pd.DataFrame(data={"id":test["id"], "sentiment":preds})
submission = pd.read_csv(config.origin_submission_path)
# preds = model.predict(x_test)
preds = model.predict_proba(x_test)[:, 1]
submission['sentiment'] = preds
submission.to_csv("../data/ml_submission.csv",index=False)
submission.head()

,id,sentiment
0,12311_10,0.966827
1,8348_2,0.020690
2,5828_4,0.496047
3,7186_2,0.656353
4,12128_7,0.947959


In [9]:
# print(submission.shape)
# print(submission[submission['sentiment'] == 1].shape)